In [3]:
import  requests
import json

In [29]:
headers = {
  "Accept": "application/json",
  "app_id": "8beabadc",
  "app_key": "4d85f3e2e7cd293da9a811f156c99841"
}

In [30]:
word_id = 'love'

In [51]:
url = 'https://od-api.oxforddictionaries.com/api/v2/thesaurus/en/' + word_id + '?strictMatch=true'
r = requests.get(url, headers = {'app_id' : app_id, 'app_key' : app_key})


In [52]:
t={}
t=json.dumps(r.json())
d=json.loads(t)

In [53]:
d

{'id': 'love',
 'metadata': {'operation': 'retrieve',
  'provider': 'Oxford University Press',
  'schema': 'Thesaurus'},
 'results': [{'id': 'love',
   'language': 'en-gb',
   'lexicalEntries': [{'entries': [{'senses': [{'antonyms': [{'language': 'en',
           'text': 'hatred'}],
         'examples': [{'text': 'his friendship with Helen grew into love'},
          {'text': 'she has a great love for her children'}],
         'id': 't_en_gb0008942.001',
         'subsenses': [{'id': 'id9fdbe2d5-b939-4de3-9ac3-8f9eb894d5e9',
           'synonyms': [{'language': 'en', 'text': 'devotion'},
            {'language': 'en', 'text': 'adoration'},
            {'language': 'en', 'text': 'doting'},
            {'language': 'en', 'text': 'idolization'},
            {'language': 'en', 'text': 'worship'}]},
          {'id': 'id88d99594-154a-4cc4-95e0-8df5867e76d7',
           'synonyms': [{'language': 'en', 'text': 'passion'},
            {'language': 'en', 'text': 'ardour'},
            {'language